In [1]:
import napari

In [2]:
# create the viewer and window
viewer = napari.Viewer()

In [3]:
import numpy as np

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def signed_angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    signed = 1
    if v1_u[0]*v2_u[1] - v1_u[1]*v2_u[0] < 0: 
        signed = -1
    return signed * np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

def get_angle(a, ref):
    return np.apply_along_axis(signed_angle_between, 1, a, ref)

In [13]:
def optical_flow_vector_field(path_in, path_out, slices, radius):
        
    from tifffile import imread, imwrite
    from pathlib import Path
    import numpy as np
    from skimage.registration import optical_flow_ilk

    nvec = 20  # Number of vectors to be displayed along each image dimension
    
    path_in = Path(path_in)
    path_out = Path(path_out)
    
    #load raw image
    image = imread(path_in).astype("uint8")
    im = image[slices]

    norms = np.zeros(im.shape)
    vectors = np.zeros(((len(im)-1)*((nvec+1)**2), 2, 3))
    timepoints = np.array(list(range(len(image))))[slices]
    for t in range(len(im)-1):
        print(t)
        # --- Compute the optical flow
        v, u = optical_flow_ilk(image[timepoints[t]], image[timepoints[t+1]], radius=radius)

        # --- Compute flow magnitude
        norms[t] = np.sqrt(u ** 2 + v ** 2)
        

        nl, nc = im[t].shape
        step = max(nl//nvec, nc//nvec)
        
        y, x = np.mgrid[:nl:step, :nc:step]
        u_ = u[::step, ::step]
        v_ = v[::step, ::step]
        
        time_point = np.ones((nvec+1)**2) * t
        time_point_fix = np.zeros((nvec+1)**2)
        start = np.array(list(zip(time_point, y.ravel(),x.ravel())))
        end = np.array(list(zip(time_point_fix, v_.ravel(),u_.ravel())))

        vectors[t*len(start):(t+1)*len(start),0,:] = start
        vectors[t*len(start):(t+1)*len(start),1,:] = end
    
    angles = get_angle(vectors[:,1,1:], np.array([1, 0]))

    data_vector = (u, v, u_, v_)
    return im, norms, vectors, angles, data_vector

path_in = '../data/placozoan-movie.tif'
path_out = '../data/placozoan_vector.tif'

im, norms, vectors, angles, data_vector = optical_flow_vector_field(path_in, path_out, slice(10, 100, 10), 15)


0
1
2
3
4
5
6
7


In [ ]:
np.save("../data/results/vector_opticflow.npy", data_vector) # save data

In [14]:
# create the features dictionary.
features = {
    'angle': angles,
}

viewer.add_image(im)
viewer.add_image(norms, colormap = 'viridis')

# add the vectors
layer = viewer.add_vectors(
    vectors,
    edge_width=3,
    features=features,
    edge_color='angle',
    edge_colormap='hsv',
    name='vectors'
)

# set the edge color mode to colormap
layer.edge_color_mode = 'colormap'